In [ ]:
#CHUNK OF CODE FOR THE LAND SURFACE TEMP CALC -- INCLUDES NDVI; WILL BE APPENDED TO OUR CBG TABLE FOR NDVI, 
#LAND SURFACE TEMP 

import ee 
import folium
import geehydro
import json
import requests
from datetime import datetime as dt
from IPython.display import Image

ee.Initialize()

maricopa = [[-113.6318726643051,32.3815643902006],
            [-110.8797974689926,32.3815643902006],
            [-110.8797974689926,34.413007528786],
            [-113.6318726643051,34.413007528786],
            [-113.6318726643051,32.3815643902006]] 
  
maricopa_AOI = ee.Geometry.Polygon(maricopa)


# polygon for where phoenix  is
poly = [[-112.35608531078948,33.22139301900224], 
        [-111.89740611157073,33.22139301900224],  
        [-111.89740611157073,33.93760032210363], 
        [-112.35608531078948,33.93760032210363], 
        [-112.35608531078948,33.22139301900224]] 

phoenix_AOI = ee.Geometry.Polygon(poly)


maricopa_AOI = ee.Geometry.Polygon(maricopa)
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
landsat_AOI = landsat.filterBounds(maricopa_AOI).filterDate('2020-12-01','2021-03-31').filterMetadata('CLOUD_COVER', 'less_than', 1.5)

phoenix = landsat_AOI.mean().clip(maricopa_AOI)

parameters = {'min': 0,
              'max': 3000,
              'scale':30,
              'bands': ['B5', 'B6', 'B4'],
              'gamma': 1.4,
              'crs': 'EPSG:3857',
              'maxPixels': 10e13,
              'region': maricopa_AOI}

parameters2 = {'min': 0,
              'max': 3000,
              'scale':30,
              'bands': ['B4', 'B3', 'B2'],
              'gamma': 1.4,
              'crs': 'EPSG:3857',
              'maxPixels': 10e13,
              'region': maricopa_AOI}

ndvi = phoenix.normalizedDifference(['B5', 'B4'])
palette_ndvi = ['red', 'yellow', 'green']
ndvi_parameters = {'min': -1,
                   'max': 1,
                   'scale':30,
                   'palette': palette_ndvi,
                   'crs': 'EPSG:3857',
                   'region': maricopa_AOI}


thermal = phoenix.select('B10').multiply(0.1)
palette_thermal = ['blue','white', 'green']
thermal_parameters = {'min': 291.918,
                      'max': 302.382,
                      'palette': palette_thermal,
                      'crs': 'EPSG:3857',
                      'region': maricopa_AOI}


min = ee.Number(ndvi.reduceRegion(reducer = ee.Reducer.min(), 
                        geometry = maricopa_AOI,
                        scale = 30,
                        maxPixels = 1e9)).getInfo()['nd']

max = ee.Number(ndvi.reduceRegion(reducer = ee.Reducer.max(), 
                        geometry = maricopa_AOI,
                        scale = 30,
                        maxPixels = 1e9)).getInfo()['nd']



fractional_veg = (ndvi.subtract(min).divide(max-min)).pow(ee.Number(2))

a = 0.004
b = 0.986

emissivity = fractional_veg.multiply(a).add(b).rename("EMM")
print(emissivity.getInfo())

LST = thermal.expression(
    '(Tb/(1+(0.00115*(Tb/1.438))*log(Ep)))-273.15', {
        'Tb': thermal.select('B10'),
        'Ep': emissivity.select('EMM'),
    })

LST_palette = ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
               '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
               '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
               'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
               'ff0000', 'de0101', 'c21301', 'a71001', '911003']

LST_Params = {'min': 20,
              'max': 50,
              'scale':30,
              'palette': LST_palette,
              'crs': 'EPSG:3857',
              'region': maricopa_AOI}



In [47]:
import ee 
import folium
import geehydro
import json
import requests
from datetime import datetime as dt
from IPython.display import Image
from pandas.io.json import json_normalize
import pandas as pd

ee.Initialize()

##ADDING CBG DATA
#creating jsons for CBG, and a filter CBG that we pass back into the CBG
CBG = json.load(open("Census_Block_Group_(TIGER_Line)_-_Arizona_(2010)_.geojson"))
Duplicate_CBG = json.load(open("Census_Block_Group_(TIGER_Line)_-_Arizona_(2010)_.geojson"))

#trimming values so we can add to map
for i in range(len(Duplicate_CBG['features'])):
    value = Duplicate_CBG['features'][i]['properties']['STATEFP10'] + Duplicate_CBG['features'][i]['properties']['COUNTYFP10'] + Duplicate_CBG['features'][i]['properties']['TRACTCE10']
    CBG['features'][i]['properties']['GEOID10'] = value
    
tract_array = []  
phoenix_CBGs = []

#turning the valid tracts into an index we search for if the tracts are in maricopa
tracts = open("TRACTS.txt")
for line in tracts:
    key = line.split(";")[1]
    tract_array.append(key)

#checking to see if the CBG is in maricopa, if it is, we add it to our list of CBGs to include in map
for i in range(len(CBG['features'])):
    for j in range(len(tract_array)):
        if tract_array[j].strip() == CBG['features'][i]['properties']['GEOID10']:
            phoenix_CBGs.append(CBG['features'][i])
            
#creating a json file from our CBGs in maricopa            
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = phoenix_CBGs



distance_array = []
CBG_name = []
area_array = []
coord_array = []
phoenix = ee.Geometry.Point([-112.0740,33.4484])

for CBG in range(len(new_json['features'])):
    poly = ee.Geometry.Polygon(coords = new_json['features'][CBG]['geometry']['coordinates'])
    coord_array.append(new_json['features'][CBG]['geometry']['coordinates'])
    centroid = ee.Geometry.Point(poly.centroid().getInfo()['coordinates'])
    distance = centroid.distance(right = phoenix)
    distance_array.append(distance.getInfo()) 
    CBG_name.append(new_json['features'][CBG]['properties']['FID'])
    area_array.append(new_json['features'][CBG]['properties']['Shape__Area'])



In [48]:
import pandas

df = pandas.DataFrame(columns = ["FID"])

df["FID"] = CBG_name
df["coords"] = coord_array
df["dist_center"] = distance_array
df["CBG_area"] = area_array


,FID,coords,dist_center,CBG_area
0,1,"[[[-112.279330069562, 33.5934361259857], [-112...",24994.221844,7.223750e+05
1,2,"[[[-112.281294069861, 33.5923301257325], [-112...",25487.626624,1.743686e+06
2,3,"[[[-112.281343069012, 33.5809451241905], [-112...",24783.110299,9.933480e+05
3,4,"[[[-112.280113069741, 33.5937841259914], [-112...",24577.483248,7.488106e+05
4,5,"[[[-112.272615067796, 33.5873421255163], [-112...",24118.565379,9.447117e+05
...,...,...,...,...
2500,4174,"[[[-112.191163061356, 33.7117291466493], [-112...",30423.972486,2.107494e+06
2501,4175,"[[[-112.178905057923, 33.6980461454517], [-112...",30505.137669,3.168063e+06
2502,4176,"[[[-112.158349055585, 33.7199911495129], [-112...",30510.454927,2.399271e+06
2503,4177,"[[[-112.158349055585, 33.7199911495129], [-112...",31233.905966,4.088453e+06


In [49]:
df.to_csv(r'C:\Users\Charlie Sturr\OneDrive\Desktop\Columbia\01. Fall_2021\Environmental\01. Final Project\df.csv')